<a href="https://colab.research.google.com/github/NithinUppara2003/Projects/blob/main/UAV_Dataset_Reading_%26%26_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'uavid-v1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1077128%2F1812990%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240219%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240219T192412Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2b47a1e8db6a6a5c58911e678ab238f909d239479bfa6f53f853f2b03dbdb174fc2ac116b1136fe6a12042d7ee8a71f7c7440cf2014acf8ae017b900b9a6fbf911cebaff86e81a967fa8dad687e5ffd92505da4677f2af0bc45136fbdc38544ffacab4795fc77b38bcac9dd9f5a18d63fac92377aaf207b2988db1cd00b13758a6b442f7e74fa719e2b03ad9bf3de10b59754a16962f82e7b553fa5b33da3819407c867ea85cf2c7ae9e31d596cc255f86815f99c95391baaceff004fb37f24d01860433e2f6574d23db01878b39b9a8859e2bcba996a280907a01cc1c6bf50b68f62c50208460011cada1d46aff02d65c1fd47375ae607752d5361239f9efbd'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


* [Data Resizing](#Resizing)
* [Label Flatten](#Label_Flattening)
* [Data Augmentation](#Augmentation)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from PIL import Image

# Test
import glob
from matplotlib.pyplot import imshow

import os
import io
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## Global Parameters
tfrecord_name = "uav_images"
train_img_path = "/kaggle/input/uavid-v1/uavid_train/seq1/Images/"
train_label_path= "/kaggle/input/uavid-v1/uavid_train/seq1/Labels/"

In [ ]:
"""train_images = glob.glob(train_img_path + '*.png')
train_labels = glob.glob(train_label_path + '*.png')
# print(images)
for image in train_images[:1]:
    img = Image.open(image)
    imshow(np.asarray(img))"""

In [ ]:
"""img = Image.open(train_images[5])
imshow(np.asarray(img))"""

In [ ]:
"""img_labelled = Image.open(train_labels[5])
imshow(np.asarray(img_labelled))"""

# Resizing

In [ ]:
def resize_all_train_images_path(train_img_path):
    """
    Direkt olarak path'den alıp boyutlarını değiştiriyoruz.
    """
    train_images = glob.glob(train_img_path + '*.png')
    for index in range(len(train_images)):
        train_images[index] = tf.keras.preprocessing.image.array_to_img(tf.image.resize(np.asarray(Image.open(train_images[index])), [256,256]))
    return train_images

In [ ]:
def resize_all_train_images_list(train_images):
    """
    Direkt olarak listeden alıp boyutlarını değiştiriyoruz.
        Bu fonksiyon tfrecord data okunduktan sonra çağırabilinir.
    """

    for index in range(len(train_images)):
        train_images[index] = tf.keras.preprocessing.image.array_to_img(tf.image.resize(np.asarray(train_images[index], [256,256])))
    # test edilecek -> return train_images

In [ ]:
train_images = resize_all_train_images_path(train_img_path=train_img_path)
label_images = resize_all_train_images_path(train_img_path=train_label_path)

In [ ]:
# Test 200 file is 256x256
total = 0
for index in range(len(train_images)):
    if train_images[index].height == 256 and train_images[0].width == 256:
        total += 1

if total==200:
    print("Tüm fotoğraflar resized edildi..")

In [ ]:
np.asarray(train_images[1]).shape

## Augmentation
[Reference](https://www.tensorflow.org/api_docs/python/tf/image#:~:text=TensorFlow%20provides%20functions%20to%20adjust,training%20set%20and%20reduce%20overfitting.)

In [ ]:
train_images[0] = tf.keras.preprocessing.image.array_to_img(tf.image.transpose(np.asarray(train_images[0])))
label_images[0] = tf.keras.preprocessing.image.array_to_img(tf.image.transpose(np.asarray(label_images[0])))

In [ ]:
label_images[0]

In [ ]:
train_images[0]

In [ ]:
train_images[0] = tf.keras.preprocessing.image.array_to_img(tf.image.random_brightness(np.asarray(train_images[0]), max_delta=0.2))
label_images[0] = tf.keras.preprocessing.image.array_to_img(tf.image.random_brightness(np.asarray(label_images[0]), max_delta=0.2)) # Label a gerek var mı?

In [ ]:
train_images[0]

In [ ]:
label_images[0]

In [ ]:
train_images[0] = tf.keras.preprocessing.image.array_to_img(tf.image.resize_with_crop_or_pad(np.asarray(train_images[0]), target_height=))
label_images[0] = tf.keras.preprocessing.image.array_to_img(tf.image.resize_with_crop_or_pad(np.asarray(label_images[0]), target_height=))

# Label_Flattening

In [ ]:
def flat_all_train_labels_path(train_label_path):
    """
    Direkt olarak path'den alıp flatten uyguluyoruz..
    """

    train_labels = resize_all_train_images_path(train_label_path)

    for index in range(len(train_labels)):
        train_labels[index] = tf.keras.preprocessing.image.array_to_img(tf.image.resize(np.asarray(train_labels[index]), [ 1, 196608]))
    return train_labels

In [ ]:
def flat_all_train_labels_list(train_label_path):
    """
    Direkt olarak list'den alıp flatten uyguluyoruz..
    """

    train_labels = resize_all_train_images_list(train_label_path)

    for index in range(len(train_labels)):
        train_labels[index] = tf.keras.preprocessing.image.array_to_img(tf.image.resize(np.asarray(train_labels[index]), [ 1, 196608]))
    return train_labels

In [ ]:
train_labels = flat_all_train_labels_path(train_label_path)

In [ ]:
print(train_labels[5])

# Tüm dataları tfRecord yapıyoruz

*write_tfrecords(train_images, label_images, name_of_tfrecords)*

In [ ]:
#buf = io.BytesIO()
#train_images[5].save(buf, format='JPEG')


In [ ]:
#byte_im = buf.getvalue()

In [ ]:
#print(byte_im)

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def write_record(train_images, train_labels, tfrecord_name):
    features = []
    # Read image raw data, which will be embedded in the record file later.
    for index in range(len(train_images)):
        buf_img = io.BytesIO()
        buf_label = io.BytesIO()

        train_images[index].save(buf_img, format='PNG')



        # temp = np.array(train_images[index])

        image_string = buf_img.getvalue()


        train_labels[index].save(buf_label, format='PNG')

        # test

        # temp = np.array(train_labels[index])

        label = buf_label.getvalue()

        # For each sample there are two features: image raw data, and label. Wrap them in a single dict.
        feature = {
            'label': _bytes_feature(label),
            'image_raw': _bytes_feature(image_string),
        }

        features.append(feature)

    # Create a `example` from the feature dict.
    tf_examples = []
    for i in range(len(train_images)):
        tf_example = tf.train.Example(features=tf.train.Features(feature=features[i]))
        tf_examples.append(tf_example)
    # tf_example = tf.train.Example(features=tf.train.Features(feature=feature))

    # Write the serialized example to a record file.
    with tf.io.TFRecordWriter(tfrecord_name + ".tfrecord") as writer:
        for index in range(len(train_images)):
            writer.write(tf_examples[index].SerializeToString())


In [ ]:
write_record(train_images, label_images, tfrecord_name)

In [ ]:
def read_record(tfrecord_name):
    # Use dataset API to import date directly from TFRecord file.
    raw_image_dataset = tf.data.TFRecordDataset(tfrecord_name + ".tfrecord")

    # Create a dictionary describing the features. The key of the dict should be the same with the key in writing function.
    image_feature_description = {
        'label': tf.io.FixedLenFeature([], tf.string),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
    }

    # Define the parse function to extract a single example as a dict.
    def _parse_image_function(example_proto):
        # Parse the input tf.Example proto using the dictionary above.
        features = tf.io.parse_single_example(example_proto, image_feature_description)
        image = tf.io.decode_image(features['image_raw'], dtype=tf.uint8)
        print("TYPE: {}\tShape: {}".format(type(image), image.shape))
        #image.set_shape([3 * 256 * 256])
        image = tf.reshape(image, [256, 256, 3])

        label = tf.io.decode_image(features['label'], dtype=tf.uint8)
        print("TYPE: {}\tShape: {}".format(type(label), label.shape))
        #label.set_shape([ 1, 196608])
        label = tf.reshape(label,[ 1, 196608, 1]) # Burası değiştirilecek.

        return image,label

    dataset = raw_image_dataset.map(_parse_image_function)
    dataset = dataset.cache()
    dataset = dataset.batch(8)

    return dataset

In [ ]:
%xmode Plain

In [ ]:
dataset = read_record(tfrecord_name)

In [ ]:
type(dataset)

In [ ]:
dataset

In [ ]:
for x,y in dataset.take(1):
    print(type(x), type(y))
    x = x.numpy()
    y = y.numpy()
    print(type(x), type(y))
    print(len(x))
    print(x[0].shape)
    print(y[0].shape)
    img_neww = tf.keras.preprocessing.image.array_to_img(x[3])
    label_neww = tf.keras.preprocessing.image.array_to_img(y[0])

In [ ]:
img_neww

In [ ]:
label_neww